# erthaos-23

In [ ]:
# Assistent für die automatische Erkennung der gesuchten Motive

"""
PARAM_label_1 = "" : Die Abfragewerte für das Zielmotiv mit entsprechende Label.
                        Die Werte von 'gesuchte_episoden können nach der in der
                        Motivliste ausgewälte Zeilen, oder ihre Bestandteilen ausgefült
                        werden und anschließend in der Anfragezeile als Anfrage mit
                        'or' und/oder 'and' Opperatoren bestimmen.

Stoppwortliste für die bestimmte Sprache.
"""

from io import StringIO
from matplotlib import text
import matplotlib.pyplot as plt
import re
import nltk
import csv
from sklearn.metrics.pairwise import cosine_similarity
import xml.etree.ElementTree as ET
import mglearn
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import pandas as pd
from hilfsfunktoinen import lade_referenz_liste_aus_xsd, parse_xml
from hilfsfunktoinen import find_similar

In [ ]:
PARAM_XML_DATABASE_NAME = 'test.xml'
ROOT_NODE = ET.parse(PARAM_XML_DATABASE_NAME).getroot()
NS = {'tei': 'http://www.tei-c.org/ns/1.0'} # 3 Namespace definieren
REFERENZ_LISTE = lade_referenz_liste_aus_xsd("kf/vmf_d1.xsd")  # passe den Pfad an

In [ ]:
# CSV datensatz erstellen

# xml datei parsen
maerchen = parse_xml(ROOT_NODE, NS, REFERENZ_LISTE, parse_labels=True)
# xml header und ballast einfuegen
maerchen_header = 'quelle,index_string,inhalt,index_binar'
maerchen = f'{maerchen_header}\n{maerchen}'

# data frame erstellen 
df_maerchen = pd.read_csv(StringIO(maerchen)).dropna()

In [ ]:
pattern = r'(?<=:)(?=a)'
indexliste = Counter(
    part
    for s in df_maerchen.index_string
    for part in re.split(pattern, s)
)
print(*indexliste, sep="\n")

In [ ]:
# Nach dem Label suchen

PARAM_label_1 = ":Menschliche_OPFERGABE_beheben:"
PARAM_label_2 = ""
PARAM_label_3 = ""

for idx, gesuchte_label in zip(df_maerchen.index, df_maerchen.index_string):
    if PARAM_label_1 in gesuchte_label:# or PARAM_label_2 in gesuchte_label:# and PARAM_label_3 in gesuchte_label:
        print(f" {gesuchte_label}")
        df_maerchen.loc[idx, 'index_binar'] = 1


In [ ]:
# nicht gewollte label a in Aggregation eliminieren

# Datensätzen die diesen a Wert nicht haben aber b und/oder c Werte besitzen können,
# werden aus der Aggregatiodatei eliminiert. Dadurch wird es verhindert, dass die Datensetzen mit
# gleichen Inhalte nicht als negative Werte entgegengesetzt werden.

PARAM_elim = "a300"
df_maerchen = df_maerchen[~((df_maerchen['index_binar'] == 1) & (~df_maerchen['index_string'].str.contains(PARAM_elim, na=False)))]

In [ ]:
# Erste Trainingsrunde

X_train = df_maerchen.inhalt  # Inhalt der Texte
y_train = df_maerchen.index_binar  # Binäre Indikatoren

# Zählen der Vorkommen jeder Binärklasse in der Spalte 'index_binar'
indexliste = Counter(df_maerchen.index_binar)

# Ausgabe der Anzahl der Vorkommen jeder Binärklasse
print(indexliste)

# NaN-Werte entfernen / ersetzen
X_train = X_train.fillna("")
PARAM_NGRAMM_RANGE = (1, 1)
PARAM_MIN_DF = 2

PARAM_USE_STOPWORDS = True
custom_stop_word_list = ['graf','grafen']

if PARAM_USE_STOPWORDS:
    nltk.download('stopwords')
    nltk_stop_words_list = nltk.corpus.stopwords.words('german')
    custom_stop_word_list = custom_stop_word_list + nltk_stop_words_list

# create the machine learning pipelines
pipe = make_pipeline(
    TfidfVectorizer(min_df=PARAM_MIN_DF, stop_words=custom_stop_word_list, ngram_range=PARAM_NGRAMM_RANGE, norm=None),
    LogisticRegression()
)

perform_regularization_sweep = False
if perform_regularization_sweep:
    param_grid = {'logisticregression__C': [0.001, 0.01, 0.1, 1, 10]}
    grid = GridSearchCV(pipe, param_grid, cv=5)
    grid.fit(X_train, y_train)
    
    print("Bester Score aus der Kreuzvalidierung: {:.2f}".format(grid.best_score_))
    estimator = grid.best_estimator_
else:
    pipe.fit(X_train, y_train)
    estimator = pipe

# Extrahieren des TF-IDF-Vektorisierers aus dem besten Schätzschritt des Gittersuchlaufs
vectorizer = estimator.named_steps["tfidfvectorizer"]

In [ ]:
# Ermitteln des maximalen TF-IDF-Werts für jedes Merkmal
max_value = vectorizer.transform(X_train).max(axis=0).toarray().ravel()

# Sortieren der Merkmale nach ihren TF-IDF-Werten
sorted_by_tfidf = max_value.argsort()

# Extrahieren der Merkmalsnamen aus dem Vektorisierer
feature_names = np.array(vectorizer.get_feature_names_out())
print("Gesamte Anzahl der Merkmale: {}".format(len(vectorizer.vocabulary_)))

# Alle Koeffizienten
coefficients = estimator.named_steps["logisticregression"].coef_[0]

In [ ]:
# größter Koeffizient
max_coef = np.max(coefficients)

# Der Schwellenwert (30% empfohlen)
threshold = 0.3 * max_coef

# Positionen (Indizes), die >= Schwellenwert sind
sw_position = np.where(coefficients >= threshold)[0]

# Zugehörige Feature-Namen und Koeffizienten
sw_features = feature_names[sw_position]
sw_coefs = coefficients[sw_position]

# Indizes für absteigende Sortierung der Koeffizienten
sort_idx = np.argsort(sw_coefs)[::-1]  # ::-1 dreht die Reihenfolge um

# Absteigend sortierte Features und Koeffizienten
sw_features_sorted = sw_features[sort_idx]
sw_coefs_sorted = sw_coefs[sort_idx]

# Ausgabe nummeriert ohne Anführungszeichen

for i, feature in enumerate(sw_features_sorted):
    print(f"{i}. {feature}  ({sw_coefs_sorted[i]:.4f})")
    selected_features_text = " ".join({feature})
    
selected_features = []

for feature in sw_features_sorted:
    selected_features.append(feature)
selected_features_text = " ".join(selected_features)
# print(selected_features_text)

In [ ]:
maerchen_nicht_gelabelt = parse_xml(ROOT_NODE, NS, REFERENZ_LISTE, parse_labels=False)
maerchen_nicht_gelabelt_header = 'quelle,inhalt,index_binar'
maerchen_nicht_gelabelt = f'{maerchen_nicht_gelabelt_header}\n{maerchen_nicht_gelabelt}'
df_maerchen_nicht_gelabelt = pd.read_csv(StringIO(maerchen_nicht_gelabelt)).dropna()
df_maerchen_nicht_gelabelt = df_maerchen_nicht_gelabelt.sort_values('inhalt')

df_maerchen_nicht_gelabelt = df_maerchen_nicht_gelabelt[~(df_maerchen_nicht_gelabelt['inhalt'] == '')]

t_corpus = df_maerchen_nicht_gelabelt['inhalt'].to_list()
t_corpus.append(selected_features_text)

all_coef_vector_unlabeled = vectorizer.fit_transform(t_corpus)

model_vector_unlabeled = all_coef_vector_unlabeled[-1]
result_coefs = all_coef_vector_unlabeled[:-1]

sorted_index_unlabeled, sorted_similarities_unlabeled = find_similar(model_vector_unlabeled, result_coefs)
sorted_similarities_unlabeled = np.round(sorted_similarities_unlabeled, 5)

for i, (bewertung, quelle, inhalt) in enumerate(
    zip(
        sorted_similarities_unlabeled,
        df_maerchen_nicht_gelabelt['quelle'].iloc[sorted_index_unlabeled],
        df_maerchen_nicht_gelabelt['inhalt'].iloc[sorted_index_unlabeled]
    )
):
    print(f'{np.round(bewertung,4):9} || {quelle} || {inhalt}\n')
    if i > 40:
        break